## Working with a real world data-set using SQL and Python

### Chicago Public Schools - Progress Report Cards (2011-2012)

In [2]:
# ativa o sql magic no jupyter notebook
%load_ext sql

In [6]:
# base de dados ativa
%sql sqlite:///Chicago_Public_Schools.db

### Store the dataset in a Table

In [8]:
# importa a base de dados 
import pandas
df = pandas.read_csv('https://data.cityofchicago.org/api/views/9xs2-f89t/rows.csv?accessType=DOWNLOAD')

In [11]:
df.shape

(566, 79)

In [20]:
df_reduzido=df.iloc[0:100,0:5]

In [21]:
df_reduzido.shape

(100, 5)

In [22]:
import sqlite3
con = sqlite3.connect("Chicago_Public_Schools.db")
cur = con.cursor()

# carrega a informação na base de dados
df_reduzido.to_sql("Chicago_Public_Schools", con, if_exists='replace', index=False,method="multi")

### Query the database system catalog to retrieve table metadata

In [25]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///Chicago_Public_Schools.db
   sqlite:///RealWorldData.db
Done.


name
Chicago_Public_Schools


### Query the database system catalog to retrieve column metadata

In [26]:
%sql SELECT count(name) FROM PRAGMA_TABLE_INFO('Chicago_Public_Schools');

 * sqlite:///Chicago_Public_Schools.db
   sqlite:///RealWorldData.db
Done.


count(name)
5


### How many Elementary Schools are in the dataset?

In [28]:
%sql SELECT name FROM PRAGMA_TABLE_INFO('Chicago_Public_Schools');

 * sqlite:///Chicago_Public_Schools.db
   sqlite:///RealWorldData.db
Done.


name
School ID
Name of School
"Elementary, Middle, or High School"
Street Address
City


In [33]:
%%sql 
select count(*) as N_Schools
from 'Chicago_Public_Schools' 
where "Elementary, Middle, or High School"='ES';

 * sqlite:///Chicago_Public_Schools.db
   sqlite:///RealWorldData.db
Done.


N_Schools
78


In [39]:
df_Elementary=df.loc[df["Elementary, Middle, or High School"]=='ES', :]
df_Elementary.shape[0]

462

In [38]:
rows_count = len(df_Elementary.index)
rows_count

462

### What is the highest Safety Score?

In [42]:
df_safety_score_max=df['Safety Score'].max()
df_safety_score_max

99.0

In [64]:
# carrega a informação na base de dados
df.loc[0:490,['Name of School','Safety Score']].to_sql('Safety Score', con, if_exists='replace', index=False,method="multi")

In [65]:
%%sql
select max("Safety Score")
from "Safety Score"

 * sqlite:///Chicago_Public_Schools.db
   sqlite:///RealWorldData.db
Done.


"max(""Safety Score"")"
99.0


### Which schools have highest Safety Score?

In [72]:
# obter o index -> apenas retorna o primeiro
print(df["Safety Score"].idxmax())

11


In [81]:
df.iloc[11,1]

'Abraham Lincoln Elementary School'

In [77]:
df["Name of School"].loc[df['Safety Score']==99.0,]

11                    Abraham Lincoln Elementary School
17            Northside College Preparatory High School
33              Alexander Graham Bell Elementary School
49                        Oriole Park Elementary School
151    Mary E Courtenay Elementary Language Arts Center
203                    Ellen Mitchell Elementary School
210         Stephen Decatur Classical Elementary School
239          James E McDade Elementary Classical School
251                      Norwood Park Elementary School
280         Edgar Allan Poe Elementary Classical School
285        Annie Keller Elementary Gifted Magnet School
319                    James G Blaine Elementary School
322                 Augustus H Burley Elementary School
330                            Talman Elementary School
466                         Edgebrook Elementary School
488               Northside Learning Center High School
505                 LaSalle Elementary Language Academy
520                         Sauganash Elementary

In [66]:
%%sql
select "Name of School"
from "Safety Score"
where "Safety Score"=99.0

 * sqlite:///Chicago_Public_Schools.db
   sqlite:///RealWorldData.db
Done.


Name of School
Abraham Lincoln Elementary School
Northside College Preparatory High School
Alexander Graham Bell Elementary School
Oriole Park Elementary School
Mary E Courtenay Elementary Language Arts Center
Ellen Mitchell Elementary School
Stephen Decatur Classical Elementary School
James E McDade Elementary Classical School
Norwood Park Elementary School
Edgar Allan Poe Elementary Classical School


In [67]:
%%sql
select "Name of School"
from "Safety Score"
where "Safety Score" in (select max("Safety Score")
from "Safety Score")

 * sqlite:///Chicago_Public_Schools.db
   sqlite:///RealWorldData.db
Done.


Name of School
Abraham Lincoln Elementary School
Northside College Preparatory High School
Alexander Graham Bell Elementary School
Oriole Park Elementary School
Mary E Courtenay Elementary Language Arts Center
Ellen Mitchell Elementary School
Stephen Decatur Classical Elementary School
James E McDade Elementary Classical School
Norwood Park Elementary School
Edgar Allan Poe Elementary Classical School


### What are the top 10 schools with the highest "Average Student Attendance"?

In [91]:
df[["Name of School", "Average Student Attendance"]].sort_values(by=['Average Student Attendance'], ascending=False).head(10)

,Name of School,Average Student Attendance
500,John Charles Haines Elementary School,98.4
376,James Ward Elementary School,97.8
280,Edgar Allan Poe Elementary Classical School,97.6
96,Rachel Carson Elementary School,97.6
461,Orozco Fine Arts & Sciences Elementary School,97.6
285,Annie Keller Elementary Gifted Magnet School,97.5
6,Lenart Elementary Regional Gifted Center,97.4
57,Andrew Jackson Elementary Language Academy,97.4
346,Disney II Magnet School,97.3
191,John H Vanderpoel Elementary Magnet School,97.2


In [82]:
# carrega a informação na base de dados
df.loc[0:490,['Name of School','Average Student Attendance']].to_sql('Average Student Attendance', con, if_exists='replace', index=False,method="multi")

In [85]:
%%sql
select "Name of School", "Average Student Attendance"
from "Average Student Attendance"
order by "Average Student Attendance" desc
limit 10;

 * sqlite:///Chicago_Public_Schools.db
   sqlite:///RealWorldData.db
Done.


Name of School,Average Student Attendance
James Ward Elementary School,97.8
Rachel Carson Elementary School,97.6
Edgar Allan Poe Elementary Classical School,97.6
Orozco Fine Arts & Sciences Elementary School,97.6
Annie Keller Elementary Gifted Magnet School,97.5
Lenart Elementary Regional Gifted Center,97.4
Andrew Jackson Elementary Language Academy,97.4
Disney II Magnet School,97.3
John H Vanderpoel Elementary Magnet School,97.2
Mark Skinner Elementary School,97.1


### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance

In [92]:
%%sql
select "Name of School", "Average Student Attendance"
from "Average Student Attendance"
order by "Average Student Attendance" 
limit 5;

 * sqlite:///Chicago_Public_Schools.db
   sqlite:///RealWorldData.db
Done.


Name of School,Average Student Attendance
Velma F Thomas Early Childhood Center,None
Richard T Crane Technical Preparatory High School,57.9
Barbara Vick Early Childhood & Family Center,60.9
Dyett High School,62.5
Wendell Phillips Academy High School,63.0
